# Test for Utilities in `ContextCite`

We will use the `ContextCiter` class to attribute models' responses to sources within the context we provide to them.

In [1]:
import os
import torch

In [2]:
# for contextcite-custom
os.chdir('..')  # Move up one directory level
print(os.getcwd())  # Print the current working directory to confirm the change

/root/autodl-tmp/context-cite


In [3]:
from context_cite import ContextCiter
cache_dir = "/root/autodl-tmp/.cache/huggingface/transformers"

model_kwargs = {
    "cache_dir": cache_dir,
    "torch_dtype": torch.bfloat16,  # torch.bfloat16 / torch.float32
}

tokenizer_kwargs = {
    "cache_dir": cache_dir,
}

model_name_or_path = "meta-llama/Llama-3.2-3B-Instruct"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Example 1 (line `3` in MedQA-USMLE-test)

- This test example heavily relies on information in context, so should have high attribution scores

In [4]:
context = """
Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise. He has type 2 diabetes mellitus and osteoarthritis of the hips. Prior to admission, his medications were insulin and naproxen. He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention. His temperature is 38\u00b0C (100.4\u00b0F), pulse is 93\/min, and blood pressure is 125\/85 mm Hg. Examination shows mottled, reticulated purplish discoloration of the feet. Laboratory studies show:\nHemoglobin count 14 g\/dL\nLeukocyte count 16,400\/mm3\nSegmented neutrophils 56%\nEosinophils 11%\nLymphocytes 31%\nMonocytes 2%\nPlatelet count 260,000\/mm3\nErythrocyte sedimentation rate 68 mm\/h\nSerum\nUrea nitrogen 25 mg\/dL\nCreatinine 4.2 mg\/dL\nRenal biopsy shows intravascular spindle-shaped vacuoles.
"""

# NOTE: formatting requirement omitted: Conclude your answer with: "Therefore, the final answer is ...".
# it encourages the model to only analyze the (self-identified) correct answer, and don't spend tokens on other options
# but NOT needed during ctx attribution; only needed during final evaluation
query = """
Which of the following is the correct next action for the resident to take?

A. Renal papillary necrosis
B. Cholesterol embolization
C. Eosinophilic granulomatosis with polyangiitis
D. Polyarteritis nodosa
"""

### Example 2 (line `11` in MedQA-USMLE-test)

- This test example barely relies on information in context, so should have low attribution scores

In [18]:
context = """
A 24-year-old G2P1 woman at 39 weeks\u2019 gestation presents to the emergency department complaining of painful contractions occurring every 10 minutes for the past 2 hours, consistent with latent labor. She says she has not experienced vaginal discharge, bleeding, or fluid leakage, and is currently taking no medications. On physical examination, her blood pressure is 110\/70 mm Hg, heart rate is 86\/min, and temperature is 37.6\u00b0C (99.7\u00b0F). She has had little prenatal care and uses condoms inconsistently. Her sexually transmitted infections status is unknown. As part of the patient\u2019s workup, she undergoes a series of rapid screening tests that result in the administration of zidovudine during delivery. The infant is also given zidovudine to reduce the risk of transmission. A confirmatory test is then performed in the mother to confirm the diagnosis of HIV.
"""

query = """
Which of the following is most true about the confirmatory test?

A. It is a Southwestern blot, identifying the presence of DNA-binding proteins
B. It is a Northern blot, identifying the presence of RNA
C. It is a Northern blot, identifying the presence of DNA
D. It is an HIV-1\/HIV2 antibody differentiation immunoassay
"""

### The `ContextCiter` class

We can directly instantiate the `ContextCiter` class with a huggingface-style `pretrained_model_name_or_path`, together with a `context`, and a `query` (passed in as strings).

In [ ]:
cc = ContextCiter.from_pretrained(
    model_name_or_path, 
    context=context, 
    query=query,
    model_kwargs=model_kwargs,
    tokenizer_kwargs=tokenizer_kwargs,
)

Alternatively, we can pass in a `model` and a `tokenizer`, which are instantiated from the `huggingface` library:

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    **tokenizer_kwargs,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    **model_kwargs,
)
model.to("cuda")
cc = ContextCiter(
    model, 
    tokenizer, 
    context=context, 
    query=query,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO] Initializing `ContextCiter` from local customized context_cite


The `response` property of the ContextCiter class contains the response generated by the model. It is lazily generated when you access it.

In [6]:
print(cc.response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Based on the patient's symptoms and laboratory results, the correct next action for the resident to take is:

B. Cholesterol embolization

The patient's presentation of decreased urinary output, malaise, and elevated erythrocyte sedimentation rate (ESR) after a recent cardiac catheterization with stenting is highly suggestive of cholesterol embolization syndrome (CES). CES is a condition that occurs when cholesterol crystals are dislodged from the atherosclerotic plaques in the arterial system and embolized to the kidneys, leading to acute kidney injury (AKI).

The renal biopsy findings of intravascular spindle-shaped vacuoles are consistent with CES, which is characterized by the presence of these vacuoles in the renal vessels.

The other options are less likely:

A. Renal papillary necrosis is a condition that typically presents with severe pain and hematuria, but it is not directly related to the patient's recent cardiac catheterization.

C. Eosinophilic granulomatosis with polyangi

Under the hood, the `ContextCiter` class applies a chat template to the
tokenized context and query, and then uses the model to generate a response.
That response is then stored in the `response` property.

### Attributing the response to sources within the context

To attribute the entire response and present the attributions in a human-readable format, we can use the `get_attributions` method, and pass in `as_dataframe=True`, as well as `top_k` to limit the number of sources to include in the attributions.

In [14]:
results_df = cc.get_attributions(as_dataframe=True, top_k=5)   # dataframe format
results_df

Attributed: Based on the patient's symptoms and laboratory results, the correct next action for the resident to take is:

B. Cholesterol embolization

The patient's presentation of decreased urinary output, malaise, and elevated erythrocyte sedimentation rate (ESR) after a recent cardiac catheterization with stenting is highly suggestive of cholesterol embolization syndrome (CES). CES is a condition that occurs when cholesterol crystals are dislodged from the atherosclerotic plaques in the arterial system and embolized to the kidneys, leading to acute kidney injury (AKI).

The renal biopsy findings of intravascular spindle-shaped vacuoles are consistent with CES, which is characterized by the presence of these vacuoles in the renal vessels.

The other options are less likely:

A. Renal papillary necrosis is a condition that typically presents with severe pain and hematuria, but it is not directly related to the patient's recent cardiac catheterization.

C. Eosinophilic granulomatosis w

/root/autodl-tmp/context-cite/context_cite/utils.py:192: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  return df.style.applymap(lambda val: _color_scale(val, max_val), subset=["Score"])


,Score,Source
0,59.143,"Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise."
1,58.674,Renal biopsy shows intravascular spindle-shaped vacuoles.
2,3.905,Erythrocyte sedimentation rate 68 mm\/h
3,2.681,"He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention."
4,1.234,"Examination shows mottled, reticulated purplish discoloration of the feet."


`results` is a pandas styler object; to access the underlying dataframe:

Alternatively, `.get_attributions()` can return the attribution scores as a `numpy` array, where the `i`th entry corresponds to the attribution score for the `i`th source in the context.

In [12]:
results_np = cc.get_attributions(as_dataframe=False)   # numpy array format
results_np

Attributed: Based on the patient's symptoms and laboratory results, the correct next action for the resident to take is:

B. Cholesterol embolization

The patient's presentation of decreased urinary output, malaise, and elevated erythrocyte sedimentation rate (ESR) after a recent cardiac catheterization with stenting is highly suggestive of cholesterol embolization syndrome (CES). CES is a condition that occurs when cholesterol crystals are dislodged from the atherosclerotic plaques in the arterial system and embolized to the kidneys, leading to acute kidney injury (AKI).

The renal biopsy findings of intravascular spindle-shaped vacuoles are consistent with CES, which is characterized by the presence of these vacuoles in the renal vessels.

The other options are less likely:

A. Renal papillary necrosis is a condition that typically presents with severe pain and hematuria, but it is not directly related to the patient's recent cardiac catheterization.

C. Eosinophilic granulomatosis w

array([59.14268373,  0.        ,  0.        ,  2.68092501,  0.        ,
        1.2341084 ,  0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  3.90502294,
        0.        ,  0.        ,  0.        , 58.67421877])

We can then match these attributions to the sources using the `sources` property:

In [15]:
list(zip(cc.sources, results_np))[:5]

[('Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise.',
  np.float64(59.142683727040954)),
 ('He has type 2 diabetes mellitus and osteoarthritis of the hips.',
  np.float64(0.0)),
 ('Prior to admission, his medications were insulin and naproxen.',
  np.float64(0.0)),
 ('He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention.',
  np.float64(2.680925012239408)),
 ('His temperature is 38°C (100.4°F), pulse is 93\\/min, and blood pressure is 125\\/85 mm Hg.',
  np.float64(0.0))]

### Attributing parts of the response

`.get_attributions()` optionally takes in `start_idx` and `end_idx` to
attribute only a part of the response.

To make it easier to attribute parts of the response, the `ContextCiter` class
has a utility property `response_with_indices` that contains the response annotated with
the index of each word within the response. You can access this with
`cc.response_with_indices`.

In [33]:
print(cc.response_with_indices)

[0]Based [6]on [9]the [13]patient[20]'s [23]symptoms [32]and [36]laboratory [47]results[54], [56]the [60]correct [68]next [73]action [80]for [84]the [88]resident [97]to [100]take [105]is[107]:[108]

[110]B. [113]Cholesterol [125]embolization[137]

[139]The [143]patient[150]'s [153]presentation [166]of [169]decreased [179]urinary [187]output[193], [195]malaise[202], [204]and [208]elevated [217]erythrocyte [229]sedimentation [243]rate [248]([249]ESR[252]) [254]after [260]a [262]recent [269]cardiac [277]catheterization [293]with [298]stenting [307]is [310]highly [317]suggestive [328]of [331]cholesterol [343]embolization [356]syndrome [365]([366]CES[369])[370]. [372]CES [376]is [379]a [381]condition [391]that [396]occurs [403]when [408]cholesterol [420]crystals [429]are [433]dislodged [443]from [448]the [452]atherosclerotic [468]plaques [476]in [479]the [483]arterial [492]system [499]and [503]embolized [513]to [516]the [520]kidneys[527], [529]leading [537]to [540]acute [546]kidney [553]inj

For example, we can attribute a part of the response like so:

In [35]:
start, end = 0, 137     # first sentence
# start, end = 1413, 1584 # last sentence. NOTE: largely depend on previously generated text, thus don't indicate context attribution
cc.get_attributions(start_idx=start, end_idx=end, as_dataframe=True, top_k=5)

Attributed: Based on the patient's symptoms and laboratory results, the correct next action for the resident to take is:

B. Cholesterol embolization


/root/autodl-tmp/context-cite/context_cite/utils.py:192: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  return df.style.applymap(lambda val: _color_scale(val, max_val), subset=["Score"])


,Score,Source
0,2.964,"Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise."
1,0.928,He has type 2 diabetes mellitus and osteoarthritis of the hips.
2,0.911,"Examination shows mottled, reticulated purplish discoloration of the feet."
3,0.662,"His temperature is 38°C (100.4°F), pulse is 93\/min, and blood pressure is 125\/85 mm Hg."
4,0.460,Laboratory studies show:
